# Substitution Ciphers

In [ ]:
# Ensure we can import from src/ when running the notebook from notebooks/
import sys, os
from pathlib import Path

# repo root = parent of the notebooks folder
ROOT = Path(os.getcwd()).resolve().parent
sys.path.insert(0, str(ROOT / "src"))

# sanity check (optional)
print("cwd:", os.getcwd())
print("src on sys.path:", any(p.endswith(os.sep + "src") for p in sys.path))

In [ ]:
import string
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import pickle
from crypto.paths import DATA_DIR

## Introduction

This report examines the design, implementation, and security implications of two fundamental cryptographic techniques: the Caesar Cipher and the Simple Substitution Cipher. These methods provide insight into early encryption strategies and serve as a foundation for understanding modern cryptographic principles. This report includes encryption, decryption, and attack methods.

## Caesar Cipher

The Caesar Cipher is a shift cipher that replaces each letter in the plaintext with another letter a fixed number of positions down the alphabet. The provided implementation (`caesar_encrypt`) performs this shifting mechanism efficiently.

### Encryption

Definig a function that iterates through each character in the plaintext, shifts it according to the given key, and reconstructs the modified ciphertext while preserving non-alphabetic characters.

In [ ]:
def caesar_encrypt(plaintext: str, shift: int = 0) -> str:
    alphabet = string.ascii_lowercase
    out = []
    for ch in plaintext:
        if ch in alphabet:
            idx = (alphabet.index(ch) + shift) % 26
            out.append(alphabet[idx])
        else:
            # leave uppercase / punctuation / spaces unchanged
            out.append(ch)
    return "".join(out)

In [ ]:
# code snippet to test the implementation of the encryption function
plaintext = 'hello!' 
ciphertext = caesar_encrypt(plaintext, shift=4)

print(plaintext, '->', ciphertext) # expected output 'hello! -> lipps!'

### Decryption

Defining a function that reverses the encryption process by shifting the characters back to their original positions, restoring the plaintext.

In [ ]:
def caesar_decrypt(ciphertext: str, shift: int = 0) -> str:
    alphabet = string.ascii_lowercase
    out = []
    for ch in ciphertext:
        if ch in alphabet:
            idx = (alphabet.index(ch) - shift) % 26
            out.append(alphabet[idx])
        else:
            out.append(ch)
    return "".join(out)

In [ ]:
# code snippet to test the implementation of the decryption function
ciphertext = 'lipps!' # 'hello!' encoded with shift=4
plaintext = caesar_decrypt(ciphertext, shift=4)

print(ciphertext, '->', plaintext)  # expected output 'lipps! -> hello!'

### Ciphertext

In [ ]:
file_path = DATA_DIR / "ciphertext_caesar.txt"  # Input the Ciphertext file path
with open(file_path, "r", encoding="utf-8") as file:
    ciphertext = file.read().strip()    # Read the whole file and remove extra spaces

### Brute Force Attack

Given that the Caesar Cipher has only 25 possible keys, the brute force attack implemented in the notebook systematically tries all possible shifts and lets the operator evaluate which produces a meaningful result.

In [ ]:
for shift in range(26):    #implement decryption algorithm for all shifts
    plaintext = caesar_decrypt(ciphertext, shift) 
    print('__________________________________________________________________')
    print(f"{shift} : {''.join(plaintext[:100])}")

In [ ]:
shift = 18
plaintext = caesar_decrypt(ciphertext, shift) 
print(f"{shift} : {''.join(plaintext)[:200]}")

## Simple Substitution Cipher

A Simple Substitution Cipher maps each letter in the plaintext to a unique letter based on a randomized key. The program includes an implementation that creates such mappings dynamically.

### Encryption

Defining a function that substitutes each letter in the plaintext with a corresponding mapped letter from a predefined key using a dictionary structure.

In [ ]:
def substitution_encrypt(plaintext: str, mapping: dict[str, str]) -> str:
    out = []
    for ch in plaintext:
        if ch.isalpha() and ch.lower() in mapping:
            enc = mapping[ch.lower()]
            out.append(enc.upper() if ch.isupper() else enc)
        else:
            out.append(ch)
    return "".join(out)

In [ ]:
# code snippet to test the implementation of the encryption function
plaintext = 'hello!'
mapping = {'h': 'a', 'e': 'p', 'l': 'w', 'o': 'q'} 

ciphertext = substitution_encrypt(plaintext, mapping)

print(plaintext, '->', ciphertext) # expected output 'hello! -> apwwq!'

### Decryption

The decryption function (`substitution_decrypt`) reverses this process by using an inverse mapping of the key to restore the original plaintext.

In [ ]:
def substitution_decrypt(ciphertext: str, mapping: dict[str, str]) -> str:
    inv = {v: k for k, v in mapping.items()}
    out = []
    for ch in ciphertext:
        lo = ch.lower()
        if ch.isalpha() and lo in inv:
            dec = inv[lo]
            out.append(dec.upper() if ch.isupper() else dec)
        else:
            out.append(ch)
    return "".join(out)


In [ ]:
# code snippet to test the implementation of the decryption function
mapping = {'h': 'a', 'e': 'p', 'l': 'w', 'o': 'q'}  # previous mapping 
ciphertext = 'apwwq!'

plaintext = substitution_decrypt(ciphertext, mapping)

print(ciphertext, '->', plaintext)  # expected output 'apwwq! -> hello!'

### Ciphertext

In [ ]:
# Load ciphertext
file_path = DATA_DIR / "ciphertext_simple.txt"
with open(file_path, "r", encoding="utf-8") as file:
    ctext = file.read()
    print(ctext[:200])

### Frequency Analysis Attack

This cipher is vulnerable to frequency analysis, which is implemented using the Counter module to compare letter frequencies in the ciphertext against known English letter distributions.

#### English Letters Distribution

This code cell includes a letter frequency analysis function(`letter_distribution`) to evaluate the distribution of characters, leveraging known statistical patterns to deduce the mapping.

In [ ]:
# function to infer the letter distribution from a text
def letter_frequencies(text: str) -> dict[str, float]:
    """
    Return normalized frequencies of a–z in the given text (case-insensitive).
    Only includes letters that appear at least once, sorted by letter.
    """
    text = text.lower()
    filtered = [ch for ch in text if ch in string.ascii_lowercase]
    n = len(filtered)
    if n == 0:
        return {}
    counts = Counter(filtered)
    return {char: count / n for char, count in sorted(counts.items())}

In [ ]:
# code snippet to test the implementation of `letter_frequencies`
text = 'hello world!'

letter_frequencies(text)
# expected ouput: 
# {'d': 0.1, 'e': 0.1, 'h': 0.1, 'l': 0.3, 'o': 0.2, 'r': 0.1, 'w': 0.1, ...}

In [ ]:
# Load English Text
file_path = DATA_DIR / "wikipedia_cybersecurity.txt"
with open(file_path, "r", encoding="utf-8") as file:
    txt = file.read().lower()
    print(txt[:200])

In [ ]:
# estimate the English letters distribution 
txt_distribituin = letter_frequencies(txt)
print(txt_distribituin)

In [ ]:
# plot the English letter distribution
plt.bar(txt_distribituin.keys(), txt_distribituin.values(), align="center", width=0.5, alpha=0.5, color="red")
plt.title("Letter Frequency Distribution")
plt.xlabel("Letters")
plt.ylabel("Frequency")

In [ ]:
# store the distribution as a pickle file
with open("eng_dst.pkl", "wb") as f:
    pickle.dump(txt_distribituin, f)

#### Perform attack

This code generates an initial letter mapping by comparing frequency distributions of the ciphertext and the expected plaintext. It:  

1. **Computes Letter Frequencies** – Analyzes letter distribution in the ciphertext (`ciphertext_distribution`).  
2. **Sorts by Frequency** – Orders both the ciphertext and expected plaintext distributions in descending order.  
3. **Creates an Initial Mapping** – Matches the most frequent letters in the plaintext with those in the ciphertext using `zip()`.  

This provides a starting point for decrypting a **substitution cipher** using frequency analysis.

In [ ]:
# perform Frequency analysis attack
ciphertext_distribiution = letter_frequencies(ctext)
ciphertext_distribiution = {k: v for k, v in sorted(ciphertext_distribiution.items(), key=lambda x: x[1], reverse=True)}
txt_distribituin = {k: v for k, v in sorted(txt_distribituin.items(), key=lambda x: x[1], reverse=True)}
mapping = {txt_key: cipher_key for (txt_key, _), (cipher_key, _) in zip(txt_distribituin.items(), ciphertext_distribiution.items())}
fig , axes = plt.subplots(2, 1, figsize=(6, 12))
axes[0].bar(txt_distribituin.keys(), txt_distribituin.values(), align="center", width=0.5, alpha=0.5, color="blue")
axes[0].set_title("Eng Alphabet Letter Distribution")
axes[0].set_xlabel("Letters")
axes[0].set_ylabel("Frequency")
axes[1].bar(ciphertext_distribiution.keys(), ciphertext_distribiution.values(), align="center", width=0.5, alpha=0.5, color="red")
axes[1].set_title("Ciphertext Letter Distribution")
axes[1].set_xlabel("Letters")
axes[1].set_ylabel("Frequency")


In [ ]:
# print mapping
print(mapping)

In [ ]:
# print decrypted plaintext
plaintext = substitution_decrypt(ctext, mapping)
print(plaintext[:200])


#### Update the mapping

As it is obvious in the printed plaintext, the mapping isn't fully correct and it needs to be updated with the correct values. We make an educated guess on what is the actual word in the main plaintext based on the word in the obtained plaintext and update the mapping.

*exteonal links* --> *external links* --> r : o

*a knrvn ulaintext attadk* --> *a known plaintext attack* --> o : r , w : v, p : u , c : d

then we search the mapping for these values and then write the right mapping.

`new_mapping = {'r' : 's', 'o' : 'd', 'w' : 'u', 'p' : 'y', 'c' : 'p'}`

Now we must update the mapping in our program, these are the steps:

1. **Updating the Mapping** – The program takes `new_mapping`, a dictionary of corrections, and updates the existing mapping.
2. **Handling Conflicts** – If a letter in (`new_mapping`) is already assigned in `mapping`, the code swaps values to maintain uniqueness.
3. **Applying Changes** – The mapping is updated, ensuring correct letter assignments while avoiding duplicates.
4. **Decrypting Again** – The refined mapping is used to decrypt the ciphertext, bringing the output closer to the correct plaintext.
5. **Repeating Until Correct** – This process is repeated with new educated guesses until the plaintext is fully recovered.

In [ ]:
new_mapping = {'r' : 's', 'o' : 'd', 'w' : 'u', 'p' : 'y', 'c' : 'p'}

reverse_mapping = {v: k for k, v in mapping.items()}

for key, new_value in new_mapping.items():
    # Find the key that currently holds new_value
    old_key = reverse_mapping[new_value]

    # Swap values
    mapping[old_key], mapping[key] = mapping[key], new_value

    # Update reverse mapping
    reverse_mapping[mapping[old_key]] = old_key
    reverse_mapping[mapping[key]] = key
        

print(f'mapping: {mapping}\n')              
print(f'plaintext:\n {substitution_decrypt(ctext, mapping)[:200]}')

cipver --> cipher

lossless hata codpression --> lossless data compression --> d : h , m : d 

*reyerences* --> *references* --> f : y

*encrgption* --> *encryption* --> y : g

*smfstitmtion* --> *substitution* --> u : m , b : f

`new_mapping = {'m' : 'k', 'y' : 'e', 'd' : 'j' , 'f' : 'r' , 'u' : 'x' , 'b' : 'o'}`

In [ ]:
new_mapping = {'m' : 'k', 'y' : 'e', 'd' : 'j' , 'f' : 'r' , 'u' : 'x' , 'b' : 'o'}

reverse_mapping = {v: k for k, v in mapping.items()}

for key, new_value in new_mapping.items():
    # Find the key that currently holds new_value
    old_key = reverse_mapping[new_value]

    # Swap values
    mapping[old_key], mapping[key] = mapping[key], new_value

    # Update reverse mapping
    reverse_mapping[mapping[old_key]] = old_key
    reverse_mapping[mapping[key]] = key

print(f'mapping: {mapping}\n')
print(f'plaintext:\n {substitution_decrypt(ctext, mapping)[:200]}')

*unizue* --> *unique* --> q : z

`new_mapping = {'q' : 'h'}`

In [ ]:
new_mapping = {'q' : 'h'}

reverse_mapping = {v: k for k, v in mapping.items()}

for key, new_value in new_mapping.items():
    # Find the key that currently holds new_value
    old_key = reverse_mapping[new_value]

    # Swap values
    mapping[old_key], mapping[key] = mapping[key], new_value

    # Update reverse mapping
    reverse_mapping[mapping[old_key]] = old_key
    reverse_mapping[mapping[key]] = key

print(f'mapping: {mapping}\n')
print(f'plaintext:\n {substitution_decrypt(ctext, mapping)[:200]}')

## Conclusion

The provided Python implementation highlights the vulnerabilities of both the **Caesar Cipher** and **Simple Substitution Cipher**. While the Caesar Cipher is trivially broken using brute force, the Simple Substitution Cipher requires a more advanced frequency analysis attack. The results emphasize the importance of using more secure cryptographic techniques in real-world applications.

## Affine cipher

### Encryption

In [ ]:
from math import gcd

In [ ]:
def affine_encrypt(plaintext: str, a: int, b: int) -> str:
    alphabet = string.ascii_lowercase
    out = []
    for ch in plaintext:
        if ch in alphabet:
            x = ord(ch) - ord("a")
            y = (a * x + b) % 26
            out.append(chr(y + ord("a")))
        else:
            out.append(ch)
    return "".join(out)

In [ ]:
plaintext = 'hello world!'
a, b = 3, 1

ciphertext = affine_encrypt(plaintext, a, b)
print(plaintext, '->', ciphertext) # expected output 'hello world! -> wniir praik!'

### Decryption

In [ ]:
def affine_decrypt(ciphertext: str, a: int, b: int) -> str:
    # find modular inverse of a mod 26
    a_inv = None
    for i in range(26):
        if (a * i) % 26 == 1:
            a_inv = i
            break
    if a_inv is None:
        # not invertible; return text unchanged (or raise)
        return ciphertext

    alphabet = string.ascii_lowercase
    out = []
    for ch in ciphertext:
        if ch in alphabet:
            y = ord(ch) - ord("a")
            x = (a_inv * (y - b)) % 26
            out.append(chr(x + ord("a")))
        else:
            out.append(ch)
    return "".join(out)

In [ ]:
ciphertext = 'wniir praik!'
a, b = 3, 1

plaintext = affine_decrypt(ciphertext, a, b)
print(ciphertext, '->', plaintext) # expected output 'wniir praik! -> hello world!'

### Ciphertext

In [ ]:
# Load ciphertext
file_path = DATA_DIR / 'ciphertext_affine.txt'
with open(file_path, 'r') as file:
    ciphertext = file.read()

### Breaking Cipher

In [ ]:
freq = letter_frequencies(ciphertext)
freq = {k: v for k, v in sorted(freq.items(), key=lambda x: x[1], reverse=True)}
mapping = {k: v for k, v in zip( txt_distribituin.keys(), freq.keys())}
print(mapping)

In [ ]:
keys = list(mapping.keys())
x_1 , x_2 = (ord(keys[0]) - ord('a')) % 26 , (ord(keys[1]) - ord('a')) % 26
y_1 , y_2 = (ord(mapping[keys[0]]) - ord('a')) % 26, (ord(mapping[keys[1]]) - ord('a')) % 26

a = ((y_2 - y_1) * pow((x_2 - x_1), -1, 26)) % 26
b = (y_1 - a * x_1) % 26

if gcd(a, 26) == 1:
    decrypted = affine_decrypt(ciphertext, a, b)
    print(decrypted[:200])
else:
    print('a is not invertible')

## Conclusion

The Affine cipher is **insecure** because it preserves letter frequencies, making it vulnerable to **frequency analysis** and **brute force attacks** due to its small keyspace. Our Python code demonstrates how easily it can be broken, proving it's unsuitable for modern encryption.